In [ ]:
import os
import cv2
import csv
import numpy as np
from itertools import chain
from glob import glob
import pandas as pd
from google.colab import drive
from matplotlib import pyplot as plt
from scipy.ndimage import center_of_mass
from PIL import Image
drive.mount('/content/drive', force_remount=True)

In [2]:
#---------------------------DEFINING ALL FUNCTIONS------------------------------
def num_white_pixels(masks):
  white_pixels = 0
  for path in masks:
    ma = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    ma = ma / 255.0
    ma = ma > 0.5
    white_pixels += list(chain.from_iterable(ma)).count(1)
  return white_pixels

def num_red_pixels(masks):
  red_pixels = 0
  for path in masks:
    if path == '/content/drive/MyDrive/Labeled_52/031306_LGcoil_LL2_ser5.14.tif':
      continue # empty mask
    ma = Image.open(path).convert('L')
    ma = ma.resize((632, 860))
    y, x = center_of_mass(np.asarray(ma))
    ma = ma.crop((x - 200, y - 200, x + 200, y + 200))
    ma = ma.resize((224, 224))
    max = np.max(ma)
    min = np.min(ma)
    ma = (ma-min)/(max-min)
    ma = ma > 0.5
    red_pixels += list(chain.from_iterable(ma)).count(1)
  return red_pixels

def get_labeled_masks(date):
  rootdir = '/content/drive/MyDrive/Labeled_52'
  paths = glob(os.path.join(rootdir, '*'))
  return [f for f in paths if date in f]

def get_morphed_masks(date):
  rootdir = '/content/drive/MyDrive/Morphed_52/offset-2.5'
  paths = glob(os.path.join(rootdir, '*'))
  return [f for f in paths if date in f]

def get_dead_sum(date, paths):

  # Load alive mask paths
  alive_masks = [f for f in paths if date in f]

  # Load entire tumor mask paths
  entire_masks = [f.replace('Predictions', 'Morphed') for f in alive_masks]

  alive_sum = num_white_pixels(alive_masks)
  total_sum = num_white_pixels(entire_masks)
  dead_sum = total_sum - alive_sum
  return dead_sum

In [ ]:
#----------------------------GRAPHING TRAINING LOGS-----------------------------
model_name = 'no-weight'
plot = ['precision', 'val_precision']
use_all = False

df = pd.read_csv(f'/content/drive/MyDrive/Files/logs/log-{model_name}.csv')
df.plot(x = 'epoch', y = plot, title = 'Unweighted')
plt.vlines(75, 0, 1, colors = 'r', linestyles = '--')
plt.show()

In [ ]:
#----------------------------GRAPHING ERROR RATE--------------------------------

x = [4, 9, 13] # days

y_label = []
y_label.append( num_red_pixels( get_labeled_masks('031306') ) )
y_label.append( num_red_pixels( get_labeled_masks('031806') ) )
y_label.append( num_red_pixels( get_labeled_masks('032206') ) )
print(f'y_label: {y_label}')

y_morph = []
y_morph.append( num_white_pixels( get_morphed_masks('031306') ) )
y_morph.append( num_white_pixels( get_morphed_masks('031806') ) )
y_morph.append( num_white_pixels( get_morphed_masks('032206') ) )
print(f'y_morph: {y_morph}')

plt.plot(x, y_label)
plt.plot(x, y_morph)
plt.xlabel('Days')
plt.ylabel('Difference between label & morphed (in pixels')
plt.title('Error Rate for Morphed Predictions')
plt.xticks(list(range(4, 14)))
ax = plt.gca()
ax.legend(['Label', 'Morphed'])
plt.show()

In [ ]:
#---------------------------GRAPHING DEAD TUMOR VOLUME--------------------------

x = [4, 9, 13] # days


pred_path = '/content/drive/MyDrive/Predictions_52/offset-2.5'
pred = glob(os.path.join(pred_path, '*'))
y_pred = []
y_pred.append( get_dead_sum('031306', pred) )
y_pred.append( get_dead_sum('031806', pred) )
y_pred.append( get_dead_sum('032206', pred) * 1.3 )

fig, ax = plt.subplots()
ax.errorbar(x, y_pred, yerr = error, fmt='-o')
plt.xlabel('Days')
plt.ylabel('Dead Tumor Volume in pixels^3')
plt.title('Change in Dead Tumor Over Course of BBCT')
plt.xticks(list(range(4, 14)))
plt.show()